In [1]:
%%HTML

<h1>Artificial Reef Aerial Survey Results</h1>

New York State's Artificial Reefs were developed to increase fisheries habitat. They provide marine fish and other organisms additional opportunities for shelter and foraging and may increase productivity in the areas where they are placed. Anglers visit artificial reef sites to benefit from the increased fishing opportun

In [1]:
import numpy as np
import pandas as pd
import nbconvert
import datetime
import calendar

In [2]:
#graphing stuff
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
#%matplotlib inline
from mpl_toolkits.basemap import Basemap

In [3]:
#get the raw data
def reefBoatCount():
    sourceData = pd.read_csv("C:/Users/maalbino/Documents/project_dev/Reefs/ReefsAerialSurvey_boatcount.csv")
    sourceData['Date'] = pd.to_datetime(sourceData['Date'], format='%m/%d/%Y')
    sourceData.rename(columns={'Number of Boats':'NumberOfBoats','Reef ':'Reef','Boat Type':'BoatType'}, inplace=True)
    return sourceData

### Total boat counts

In [4]:
boatCount = reefBoatCount()

In [5]:
#Total number of boats counted over the duration of the survey 1995-2015
totalBoats = int(boatCount['NumberOfBoats'].sum())
#Break down by date, year and month
byDate = boatCount.groupby('Date').sum()
#byYear = boatCount.groupby(boatCount.Date.map(lambda x: x.year)).sum()
#byYear.index.names = ['Year']
#byMonth = boatCount.groupby(boatCount.Date.map(lambda x: x.month)).sum()
#byMonth.index.names = ['Month']
#Break down by reef
byReef = boatCount.groupby('Reef').sum()
#Break down by boat type
byBoatType = boatCount.groupby('BoatType').sum()

In [6]:
analysisBy = "<p>Analysis performed on 8/24/2014 by MAH for years 1995-2015 using reefSurveyResults.ipynb</p>"
print analysisBy

<p>Analysis performed on 8/24/2014 by MAH for years 1995-2015 using reefSurveyResults.ipynb</p>


In [7]:
#How many flights total
flights = len(byDate)
totalBoatsStr = "<h2>Summary</h2><p>The aerial reef survey observed " + str(totalBoats) + " boats utilizing the reefs on " + str(flights) + " flights between 1995 and 2015.</p>"
print totalBoatsStr

<h2>Summary</h2><p>The aerial reef survey observed 9382 boats utilizing the reefs on 159 flights between 1995 and 2015.</p>


In [8]:
#How many flights that saw over 100 boats?
moreThan100 = byDate[byDate.NumberOfBoats > 100]
print "Of these " + str(flights) + " flights, " + str(len(moreThan100)) + " registered greater than 100 boats."
topTen = moreThan100.sort_values(['NumberOfBoats'], ascending=False)
print "These are the ten most successful flights:"
topTen.head(n=10)
topTenTbl = topTen.head(n=10)
topTenStr = "<p>Of these " + str(flights) + " flights, " + str(len(moreThan100)) + " registered greater than 100 boats. These are the ten most successful flights:</p>"
topTenTbl

Of these 159 flights, 26 registered greater than 100 boats.
These are the ten most successful flights:


,NumberOfBoats
Date,
2005-10-02,284.0
2015-07-11,277.0
1997-09-14,256.0
2007-08-19,218.0
2004-10-04,217.0
2002-08-25,215.0
2001-06-09,199.0
2004-06-12,197.0
2001-07-07,192.0


In [9]:
#Make HTML top ten table
topTenTblhtml = topTenTbl\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')  

In [10]:
#How many boats were counted on any given year
byYear = boatCount.groupby(boatCount.Date.map(lambda x: x.year)).sum()
byYear.index.names = ['Year']
byYear

,NumberOfBoats
Year,
1995,335.0
1996,361.0
1997,763.0
2000,507.0
2001,573.0
2002,946.0
2003,1250.0
2004,835.0
2005,508.0


In [11]:
#add year and cpue column to df
flightsPerYear = reefBoatCount()
flightsPerYear = flightsPerYear.groupby('Date').sum()
flightsPerYear['Year'] = flightsPerYear.index.map(lambda x: x.year)
byYear['Flights'] = flightsPerYear.groupby('Year').count()
byYear['CPUE'] = byYear['NumberOfBoats'] / byYear['Flights']

In [12]:
byYear

,NumberOfBoats,Flights,CPUE
Year,,,
1995,335.0,8,41.875000
1996,361.0,8,45.125000
1997,763.0,13,58.692308
2000,507.0,4,126.750000
2001,573.0,10,57.300000
2002,946.0,14,67.571429
2003,1250.0,16,78.125000
2004,835.0,16,52.187500
2005,508.0,8,63.500000


In [13]:
#Double check the math
byYear['Flights'].sum()

159L

In [14]:
#make html table
byYearStr = "<h2>Boat Counts Broken Down By Year</h2>"
byYearhtml = byYear\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')
    

In [15]:
#Graph the annual results, this produces a figure as a separate file
#by year
byYear.plot.bar(title="Aerial Reef Survey Summary By Year")
#plt.show()
plt.savefig('S:/GISWorkGroup/outputs/byYearBar.png')
byYearGraphhtml = '<img src="S:/GISWorkGroup/outputs/byYearBar.png">'

In [16]:
byMonth = boatCount.groupby(boatCount.Date.map(lambda x: x.month)).sum()
byMonth.index.names = ['Month']
byMonth

,NumberOfBoats
Month,
1,0.0
4,34.0
5,899.0
6,1390.0
7,1489.0
8,1675.0
9,1177.0
10,1734.0
11,756.0


In [17]:
#add year and cpue column to df
flightsPerMonth = reefBoatCount()
flights = flightsPerMonth.groupby('Date').sum()
flights['Month'] = flights.index.map(lambda x: x.month)
#flights
byMonth['Flights'] = flights.groupby('Month').count()
byMonth['CPUE'] = byMonth['NumberOfBoats'] / byMonth['Flights']
byMonth

,NumberOfBoats,Flights,CPUE
Month,,,
1,0.0,1,0.000000
4,34.0,6,5.666667
5,899.0,19,47.315789
6,1390.0,19,73.157895
7,1489.0,19,78.368421
8,1675.0,22,76.136364
9,1177.0,16,73.562500
10,1734.0,25,69.360000
11,756.0,15,50.400000


In [18]:
#check my math
byMonth['Flights'].sum()

159L

In [19]:
byMonthStr = "<h2>Boat Counts Broken Down By Month</h2>"
byMonthhtml = byMonth\
    .to_html()\
    #.replace('<table border="1" class="dataframe">','<table class="table table-striped">')

In [20]:
#by month
byMonth.plot.bar(title="Aerial Reef Survey Summary By Month")
plt.savefig('S:/GISWorkGroup/outputs/byMonthBar.png')
byMonthGraphhtml = '<img src="S:/GISWorkGroup/outputs/byMonthBar.png">'

In [21]:
#We want to know which reef has had the greatest use over the entirety of the survey
reefs = boatCount.groupby('Reef').sum()
reefs = reefs.sort_values(['NumberOfBoats'], ascending=False)
reefs

,NumberOfBoats
Reef,
Rockaway,2209.0
Atlantic Beach,1475.0
Hempstead,1227.0
Fire Island,1057.0
Kismet,758.0
McAllister,635.0
Yellowbar,535.0
Shinnecock,418.0
Moriches,410.0


In [22]:
byReefStr = "<h2>Boat Counts Broken Down By Reef</h2>"
byReefhtml = reefs\
    .to_html()

In [23]:
reefs.plot.pie(y='NumberOfBoats', figsize=(8,8), autopct='%.1f', legend=False)
plt.savefig('S:/GISWorkGroup/outputs/byReefPie.png')
byReefGraphhtml = '<img src="S:/GISWorkGroup/outputs/byReefPie.png">'

## Break down by boat size


In [24]:
moreBoats = reefBoatCount()
bySize = moreBoats.groupby('BoatType').sum()
bySize['Percentage'] = (bySize['NumberOfBoats'] / totalBoats)*100
bySize

,NumberOfBoats,Percentage
BoatType,,
Large,664.0,7.077382
Small,8718.0,92.922618


In [ ]:
#Basic Map
reefLocations = pd.read_csv("C:/Users/maalbino/Documents/opendata/Reefs/reefPoints.csv")

## Break down by boat size

In [25]:
#Create HTML page
htmlStart = '''
<html>
    <body>
        <img src="http://internal.dec.state.ny.us/images/dpaeimages/logodecweb.png" width="400"><br>
        <font face="Arial" color="#2C5234">
        <h1>Artificial Reef Survey Results</h1> </font>
        <font face="Arial">'''

htmlMiddle1 = "\n"+ totalBoatsStr + "\n" +  topTenStr + topTenTblhtml + "<hr>" + byYearStr + byYearhtml + byYearGraphhtml+"<hr>"\
    + byMonthStr + byMonthhtml + byMonthGraphhtml +"<hr>" + byReefStr + byReefhtml + byReefGraphhtml


htmlEnd = '<font face="Arial" size="1">' + analysisBy + "</font></font></body></html>"

htmlString = htmlStart + "<hr>" + htmlMiddle1 + "<hr>" + htmlEnd
f = open('C:/Users/maalbino/Desktop/test1.html', 'w')
f.write(htmlString)
f.close()